In [13]:
import os
import shutil
import random

# Paths
dataset_path = "..\OccupantDetection\Input\data"
images_dir = os.path.join(dataset_path, 'images')
labels_dir = os.path.join(dataset_path, 'labels')

# Output directories
output_path = "..\OccupantDetection\split_dataset"
splits = ["train", "val", "test"]
for split in splits:
  os.makedirs(os.path.join(output_path, split, "images"), exist_ok=True)
  os.makedirs(os.path.join(output_path, split, "labels"), exist_ok=True)

# List all images
images = [f for f in os.listdir(images_dir) if f.endswith((".jpg", ".png"))]
filtered_images = [f for f in images if os.path.exists(os.path.join(labels_dir, os.path.splitext(f)[0] + '.txt'))]

# Limit to 5000 images
selected_images = filtered_images[:5000]
random.shuffle(selected_images)

# Split dataset
train_ratio, val_ratio = 0.7, 0.2 # 70% train, 20% val, 10% test
train_split = int(len(selected_images) * train_ratio)
val_split = int(len(selected_images) * (train_ratio + val_ratio))

train_files = selected_images[:train_split]
val_files = selected_images[train_split:val_split]
test_files = selected_images[val_split:]

# Function to move files
def move_files(file_list, split):
  img_count, label_count = 0, 0
  for file in file_list:
    img_src = os.path.join(images_dir, file)
    label_src = os.path.join(labels_dir, os.path.splitext(file)[0] + '.txt')

    img_dest = os.path.join(output_path, split, "images", file)
    label_dest = os.path.join(output_path, split, "labels", os.path.splitext(file)[0] + ".txt")

    shutil.copy(img_src, img_dest)
    img_count += 1
    if os.path.exists(label_src):
      shutil.copy(label_src, label_dest)
      label_count +=1

    print(f"{split.upper()} - Images: {img_count}, Annotations: {label_count}")

# Move files into respective directories
move_files(train_files, "train")
move_files(val_files, "val")

TRAIN - Images: 1, Annotations: 1
TRAIN - Images: 2, Annotations: 2
TRAIN - Images: 3, Annotations: 3
TRAIN - Images: 4, Annotations: 4
TRAIN - Images: 5, Annotations: 5
TRAIN - Images: 6, Annotations: 6
TRAIN - Images: 7, Annotations: 7
TRAIN - Images: 8, Annotations: 8
TRAIN - Images: 9, Annotations: 9
TRAIN - Images: 10, Annotations: 10
TRAIN - Images: 11, Annotations: 11
TRAIN - Images: 12, Annotations: 12
TRAIN - Images: 13, Annotations: 13
TRAIN - Images: 14, Annotations: 14
TRAIN - Images: 15, Annotations: 15
TRAIN - Images: 16, Annotations: 16
TRAIN - Images: 17, Annotations: 17
TRAIN - Images: 18, Annotations: 18
TRAIN - Images: 19, Annotations: 19
TRAIN - Images: 20, Annotations: 20
TRAIN - Images: 21, Annotations: 21
TRAIN - Images: 22, Annotations: 22
TRAIN - Images: 23, Annotations: 23
TRAIN - Images: 24, Annotations: 24
TRAIN - Images: 25, Annotations: 25
TRAIN - Images: 26, Annotations: 26
TRAIN - Images: 27, Annotations: 27
TRAIN - Images: 28, Annotations: 28
TRAIN - Im

In [14]:
from sys import path
import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):
  # Read classes.txt to get class names
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Create data dict
  data = {
        'path': os.path.abspath("../OccupantDetection/Input/data"),
        'train': os.path.abspath("../OccupantDetection/split_dataset/train"),
        'val': os.path.abspath("../OccupantDetection/split_dataset/val"),
        'nc': number_of_classes,
        'names': classes
    }

  # Write data to YAML file
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Define path to classes.txt and run function
path_to_classes_txt = '..\OccupantDetection\Input\data\classes.txt'
path_to_data_yaml = '..\OccupantDetection\data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!type "..\OccupantDetection\data.yaml"

Created config file at ..\OccupantDetection\data.yaml

File contents:

path: c:\Users\User\Desktop\Edge-AI\OccupantDetection\Input\data
train: c:\Users\User\Desktop\Edge-AI\OccupantDetection\split_dataset\train
val: c:\Users\User\Desktop\Edge-AI\OccupantDetection\split_dataset\val
nc: 1
names:
- Person


In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

'nvcc' is not recognized as an internal or external command,
operable program or batch file.


Using device: cuda


In [2]:
from ultralytics import YOLO
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load YOLOv8 Nano
model = YOLO("yolov8n.pt") 
model.train(data="../OccupantDetection/data.yaml", epochs=100, imgsz=640, patience=10)

Using device: cuda
Ultralytics 8.3.94  Python-3.10.0 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=../OccupantDetection/data.yaml, epochs=100, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=Fals

100%|██████████| 5.35M/5.35M [00:01<00:00, 3.86MB/s]


AMP: checks passed 


train: Scanning C:\Users\User\Desktop\Edge-AI\OccupantDetection\split_dataset\train\labels.cache... 4536 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4536/4536 [00:00<?, ?it/s]
val: Scanning C:\Users\User\Desktop\Edge-AI\OccupantDetection\split_dataset\val\labels.cache... 1784 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1784/1784 [00:00<?, ?it/s]


Plotting labels to c:\Users\User\Desktop\Edge-AI\runs\detect\train8\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to c:\Users\User\Desktop\Edge-AI\runs\detect\train8
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    100/100      4.72G      1.192     0.8148      1.229         44        640: 100%|██████████| 284/284 [00:44<00:00,  6.38it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 56/56 [00:10<00:00,  5.17it/s]


                   all       1784       7027       0.81      0.689      0.775      0.499

100 epochs completed in 1.639 hours.
Optimizer stripped from c:\Users\User\Desktop\Edge-AI\runs\detect\train8\weights\last.pt, 6.2MB
Optimizer stripped from c:\Users\User\Desktop\Edge-AI\runs\detect\train8\weights\best.pt, 6.2MB

Validating c:\Users\User\Desktop\Edge-AI\runs\detect\train8\weights\best.pt...
Ultralytics 8.3.94  Python-3.10.0 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 56/56 [00:12<00:00,  4.39it/s]


                   all       1784       7027       0.81      0.689      0.775      0.499
Speed: 0.2ms preprocess, 1.6ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to c:\Users\User\Desktop\Edge-AI\runs\detect\train8


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001C6019A6950>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [7]:
# Load trained model
model = YOLO("../OccupantDetection/runs/detect/train8/weights/best.pt")

# Perform object detection on val images and save result
results = model.predict(source="../OccupantDetection/split_dataset/val/images", save=True)



WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1784 c:\Users\User\Desktop\Edge-AI\OccupantDetection\..\OccupantDetection\split_dataset\val\images\0000f53faa4d14c3.jpg: 448x640 1 Person, 9.3ms
image 2/1784 c:\Users\User\Desktop\Edge-AI\OccupantDetection\..\OccupantDetection\split_dataset\val\images\0000fcb8ed0ea243.jpg: 448x640 3 Persons, 10.9ms
image 3/1784 c:\Users\User\Desktop\Edge-AI\OccupantDetection\..\OccupantDetection\split_dataset\val\images\0003db14e5cdc1c4.jpg: 448x640 2 Persons, 9.

In [9]:
# Display first 10 images from inference results

import glob
from IPython.display import Image, display
for image_path in glob.glob(f'runs\detect\predict2\*.jpg')[:10]:
  display(Image(filename=image_path, height=400))
  print('\n')

In [12]:
import os
import shutil

# Define directory to store model files
model_dir = r"..\OccupantDetection\my_model"
os.makedirs(model_dir, exist_ok=True)

# Copy trained model weights
src_model_path = r"..\OccupantDetection\runs\detect\train8\weights\best.pt" 
dst_model_path = os.path.join(model_dir, "my_model.pt")
shutil.copy(src_model_path, dst_model_path)

# Copy entire training directory
src_train_path = r"..\OccupantDetection\runs\detect\train8"
dst_train_path = os.path.join(model_dir, "train")
shutil.copytree(src_train_path, dst_train_path, dirs_exist_ok=True)

print("Model and training files copied successfully!")

Model and training files copied successfully!
